In [138]:
import pandas as pd
import os

payments= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\payment.csv')
orders= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\orders.csv')
cafes=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\cafe.csv')
address=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\address.csv')

merged_df=payments.merge(orders,on='payment_id',how='left',indicator=True)

payment_date_converted=pd.DatetimeIndex(merged_df['payment_date']) #convert from object to datetime payment_date_converte

merged_df['Year']=payment_date_converted.year
merged_df['Month']=payment_date_converted.month  

merged_df=merged_df.sort_values(by='payment_date',ascending=True)
merged_df=merged_df.drop_duplicates(subset='payment_id',keep='first')#removing duplicates with same payment_id
columns_todrop=['customer_id_x','emp_id','created_at','payment_type', #removing columns that are not needed
                'order_id','item_id','customer_id_y','delivery',
                'delivery_address_id','item_quantity',
                'customer_modifications','delivery_notes','_merge']
merged_df=merged_df.drop(columns=columns_todrop,axis=1)

columns=['cafe_id','payment_date','Year','Month','payment_id','amount']
merged_df=merged_df[columns]
merged_df['payment_date']=pd.to_datetime(merged_df['payment_date'])
merged_df['Year']=merged_df['Year'].astype('str')
merged_df['Month']=merged_df['Month'].astype('str')
#using .loc to change all values in column 'Month' to 2 digits
merged_df['Month'] = merged_df['Month'].str.zfill(2)
merged_df['Year_Month']=merged_df['Year']+'-'+merged_df['Month']

monthly_sales_df=merged_df.groupby(['cafe_id','Year_Month','Year','Month']).agg(
    monthly_revenue_total=('amount','sum')
).reset_index()

monthly_sales_df=monthly_sales_df.reset_index(drop=True)

cafe_address_df=pd.merge(cafes,address,on='address_id',how='left')
sales_df=pd.merge(monthly_sales_df,cafe_address_df,on='cafe_id',how='left')

sales_df=sales_df.drop(columns=['manager_emp_id','phone_number','address_id','last_update_x','address_first_line',
                         'address_id','address_second_line','postal_code','last_update_y','city'],axis=1)


#changing cafe_id 1,2,3,4,5 TO "New York","Miami","Lima","Buenos Aires","Seattle"
cafe_id_dictionary ={1: "New York", 2: "Miami", 3: "Lima",4: "Buenos Aires",5: "Seattle"}
sales_df['cafe_id'] = sales_df['cafe_id'].map(cafe_id_dictionary)
#adding column "Region" 
region_dictionary={"USA":"North America","Peru":"LATAM","Argentina":"LATAM"}
sales_df["region"]=sales_df['country'].map(region_dictionary)
sales_df=sales_df.loc[:,['cafe_id','country','region','Year_Month','Year','Month','monthly_revenue_total']]


monthly_sales_by_venue_df=sales_df

monthly_sales_by_venue_avg_df=monthly_sales_by_venue_df.groupby(['cafe_id','Month']).agg(
   avg_revenue_month_of_the_year_by_location=('monthly_revenue_total','mean')
).reset_index()
monthly_sales_by_venue_avg_df['avg_revenue_month_of_the_year_by_location']=pd.DataFrame(monthly_sales_by_venue_avg_df['avg_revenue_month_of_the_year_by_location'].apply(lambda x:round(x,2)))
monthly_sales_by_venue_df=pd.merge(monthly_sales_by_venue_df,monthly_sales_by_venue_avg_df,how='left',on=['Month','cafe_id'])
monthly_sales_by_venue_df.reset_index()

#calculating  average monthly revenue in each location 
monthly_sales_by_venue_avg_df=monthly_sales_by_venue_df.groupby(['cafe_id']).agg(
   avg_monthly_revenue_by_location=('monthly_revenue_total','mean')
).reset_index()

monthly_sales_by_venue_avg_df['avg_monthly_revenue_by_location']=pd.DataFrame(monthly_sales_by_venue_avg_df['avg_monthly_revenue_by_location'].apply(lambda x:round(x,2)))

monthly_sales_by_venue_df=pd.merge(monthly_sales_by_venue_df,monthly_sales_by_venue_avg_df,how='left',on=['cafe_id'])
monthly_sales_by_venue_df.reset_index()
monthly_sales_by_venue_df=pd.merge(monthly_sales_by_venue_df,monthly_sales_by_venue_avg_df,how='left',on=['cafe_id'])
monthly_sales_by_venue_df=monthly_sales_by_venue_df.reset_index(drop=True)
monthly_sales_by_venue_df=monthly_sales_by_venue_df.drop(columns=['avg_monthly_revenue_by_location_y'])
monthly_sales_by_venue_df=monthly_sales_by_venue_df.rename(columns={'avg_monthly_revenue_by_location_x':'avg_monthly_revenue_by_location'})
monthly_sales_by_venue_df.head()

monthly_sales_by_venue_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by cafe location\Monthly_sales_by_venue.csv',index=False)
monthly_sales_by_venue_df.to_csv(r"C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Revenue by cafe location\Monthly_sales_by_venue.csv",index=False)

Annual_sales_by_country_df=pd.DataFrame(sales_df.groupby(['country','Year'])['monthly_revenue_total'].sum())

Annual_sales_by_country_df=Annual_sales_by_country_df.reset_index()
Annual_sales_by_country_df=Annual_sales_by_country_df.rename(columns={'monthly_revenue_total':'annual_revenue_by_country'})
Total_annual_revenue_df=pd.DataFrame(Annual_sales_by_country_df.groupby(['Year']).agg(
    total_annual_revenue=('annual_revenue_by_country','sum')
)).reset_index()

Annual_sales_by_country_df=pd.merge(Annual_sales_by_country_df,Total_annual_revenue_df,how='left',on='Year')
Annual_sales_by_country_df['percentage_from_total_annual_revenue']=(Annual_sales_by_country_df['annual_revenue_by_country']/Annual_sales_by_country_df['total_annual_revenue']*100).apply(lambda x:round(x,2))
Annual_sales_by_country_df=Annual_sales_by_country_df.sort_values(by='Year')


Annual_sales_by_country_df["region"]=Annual_sales_by_country_df['country'].map(region_dictionary)

Annual_sales_by_country_df=Annual_sales_by_country_df.loc[:,['region','country','Year','annual_revenue_by_country','total_annual_revenue','percentage_from_total_annual_revenue']]


Annual_sales_by_country_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by country\Annual_sales_by_country.csv',index=False)
Annual_sales_by_country_df.to_csv(r'C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Revenue by country\Annual_sales_by_country.csv',index=False)

Annual_sales_by_region_df=pd.DataFrame(sales_df.groupby(['region','Year'])['monthly_revenue_total'].sum())
Annual_sales_by_region_df=Annual_sales_by_region_df.reset_index()
Annual_sales_by_region_df=Annual_sales_by_region_df.rename(columns={'monthly_revenue_total':'annual_revenue_by_region'})

annual_total_df=pd.DataFrame(Annual_sales_by_region_df.groupby(['Year'])['annual_revenue_by_region'].sum())
annual_total_df=annual_total_df.reset_index()
annual_total_df=annual_total_df.rename(columns={'annual_revenue_by_region':'total_annual_revenue'})

Annual_sales_by_region_df=pd.merge(Annual_sales_by_region_df,annual_total_df, on='Year',how='left')
Annual_sales_by_region_df['percentage_from_total_annual_revenue']=Annual_sales_by_region_df['annual_revenue_by_region']/Annual_sales_by_region_df['total_annual_revenue']*100
Annual_sales_by_region_df['percentage_from_total_annual_revenue']=Annual_sales_by_region_df['percentage_from_total_annual_revenue'].apply(lambda x: round(x,2))

Annual_sales_by_region_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by region\Sales_by_region_North_America_vs_LATAM.csv',index=False)
Annual_sales_by_region_df.to_csv(r'C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Revenue by region\Sales_by_region_North_America_vs_LATAM.csv',index=False)

In [106]:
Annual_sales_by_country_df.head()

,country,Year,annual_revenue_by_country,total_annual_revenue,percentage_from_total_annual_revenue
0,Argentina,2020,14651.99,69370.36,21.12
12,USA,2020,41577.66,69370.36,59.94
6,Peru,2020,13140.71,69370.36,18.94
1,Argentina,2021,17431.71,77205.21,22.58
13,USA,2021,44508.23,77205.21,57.65
